# TS-RAPOOC

### Based on RAPOOC algorithm proposed by Omar Odibat (2014)

- Step 1: Devide rows into two separate parts
- Step 2: Decide which part will be devided among all generated clusters
- Step 3: Devide the chosen cluster in columns or rows, the one that has the highest coherence
- Step 4: Check the number of clusters generated this way, if the number of clusters is reached, break, else return to step 2

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import glob
import multiprocessing
from operator import attrgetter

from metrics import *
from clusterer import *

In [2]:
%%latex
Define H for a new term 'z' in X
$$
H_{1}(I,J,X,z) =H_{0}(I,J,X) \cdot \frac{(I-1)}{(I+1)} + \frac{|2|}{(I)(I+1)} \cdot \sum_{x \epsilon X} {h(x,z,J)}
$$

<IPython.core.display.Latex object>

In [27]:
class ClusterMaps(object):
    def __init__(self, rho=None, gamma=None, H = 0):
        self.gamma = gamma
        self.rho = rho
        self.H = H
    
    def __str__(self):
        return ("rho "+ str(self.rho) + " gamma " + str(self.gamma) + " H " + str(self.H))

In [ ]:
def 

In [120]:
class TsRapooc(object):
    def __init__(self, data, n_clusters = 3):
        self._D = np.array(data)
        self._n_clusters = n_clusters
        self._labels = list()
        self._I, self._J = data.shape
        
    @property
    def n_clusters(self):
        return self._n_clusters
    
    @property
    def data(self):
        return self._D
    
    @property
    def labels(self):
        return _labels
    
    def fit(self):
        n_iteration = 0
        n_clusters = 0
        for n_iteration in range(0, self._n_clusters):
            if n_iteration == 0:
                initial_partition = BisectingClusterer(data).fit_rows()
                self._labels.append(ClusterMaps(np.where(initial_partition==0)[0],
                                   np.arange(self._J),
                                    PairBasedCoherence(self._D[np.where(initial_partition==0)[0]]).HP))
         
                
                self._labels.append(ClusterMaps(np.where(initial_partition==1)[0],
                                   np.arange(self._J),
                                   PairBasedCoherence(self._D[np.where(initial_partition==1)[0]]).HP))
            else:
                self._partition()
        
    def _partition(self):
        least_coherent = self._find_least_coherent()
        lowest_cluster = self._D[np.ix_(least_coherent.rho,least_coherent.gamma)]
        self._bisect_bicluster(lowest_cluster, least_coherent)
    
    def _find_least_coherent(self):
        return self._labels[self._labels.index(min(self._labels,key=attrgetter('H')))]
    
    def _bisect_bicluster(self,cluster, cluster_node):
        #which of the biclusters maximize coherence
        rows_map = BisectingClusterer(cluster).fit_rows()
        cols_map = BisectingClusterer(cluster).fit_cols()
        
        flg_alls= 'none'
        
        if(np.all(cols_map) or np.all(rows_map)):
            if(np.all(rows_map)):
                flg_alls = 'cols'
            else:
                flg_alls = 'rows'
            
        print(flg_alls)
        
        if(flg_alls == 'none'):
            print(cluster_node)
            print(str(rows_map))
            print(str(cols_map))
        elif (flg_alls == 'cols'):
            print(cluster_node)
            print(str(rows_map))
            print(str(cols_map))
        else:
            print()
            print(np.flatnonzero(rows_map == 0))
            print(cluster_node.rho[np.flatnonzero(rows_map == 0)])
            print(str(rows_map))
            print(str(cols_map))
                        

In [121]:
tsrapooc = TsRapooc(data)
tsrapooc.fit()

rows

[ 0  2  3  4  7  9 11 12 13 17 19 20 21 23 27 30 35 36 37 39 41 42 45 46 47
 49 50 51 52]
node
[ 0  2  3  4  8 11 13 14 15 25 27 28 31 34 43 49 54 56 58 67 71 73 78 79 80
 85 86 87 92]
[ 0.  1.  0.  0.  0.  1.  1.  0.  1.  0.  1.  0.  0.  0.  1.  1.  1.  0.
  1.  0.  0.  0.  1.  0.  1.  1.  1.  0.  1.  1.  0.  1.  1.  1.  1.  0.
  0.  0.  1.  0.  1.  0.  0.  1.  1.  0.  0.  0.  1.  0.  0.  0.  0.  1.
  1.  1.]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
rows

[ 0  2  3  4  7  9 11 12 13 17 19 20 21 23 27 30 35 36 37 39 41 42 45 46 47
 49 50 51 52]
node
[ 0  2  3  4  8 11 13 14 15 25 27 28 31 34 43 49 54 56 58 67 71 73 78 79 80
 85 86 87 92]
[ 0.  1.  0.  0.  0.  1.  1.  0.  1.  0.  1.  0.  0.  0.  1.  1.  1.  0.
  1.  0.  0.  0.  1.  0.  1.  1.  1.  0.  1.  1.  0.  1.  1.  1.  1.  0.
  0.  0.  1.  0.  1.  0.  0.  1.  1.  0.  0.  0.  1.  0.  0.  0.  0.  1.
  1.  1.]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


In [ ]:
data = pd.read_csv('TestData/SimulatedDataCoherence/BiclusterA_high.csv',header=None)
# data = (data - 0.5) * 2.0
data.describe()

In [72]:
bisect = BisectingClusterer(data)

In [73]:
bisect.fit()

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,
        1.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,
        1.,  1.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,
        1.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,
        1.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,
        0.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.])

In [74]:
bisect.fit_cols()

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.])

In [92]:
np.all([1,1,1,1])

True

rho [ 0  1  2  3  4  5  6  7  8  9 10] gamma (array([], dtype=int64),) H 0
